In [178]:
import pandas as pd
import numpy as np
from numpy import quantile, where, random
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.svm import OneClassSVM
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [179]:
df_tr_1 = pd.read_csv("C:/Users/User/github/data/TFTLCD/ph1_bytime.csv", encoding='cp949')
df_ts_1 = pd.read_csv("C:/Users/User/github/data/TFTLCD/ph2_in.csv", encoding='cp949')
df_ts_2 = pd.read_csv("C:/Users/User/github/data/TFTLCD/ph2_out.csv", encoding='cp949')

In [180]:
df_tr = df_tr_1.iloc[:, 7:50] # train
df_ts = pd.concat([df_ts_1.iloc[:,7:50], df_ts_2.iloc[:,7:50]])

In [181]:
df_tr = df_tr.replace('Normal', 0)
df_ts = df_ts.replace('정상', 0)
df_ts = df_ts.replace('이상', 1)

In [182]:
df_ts = df_ts.rename(columns={'추가정보_정상.이상여부':'Normal/Abnormal'})

In [183]:
df_tr_x = df_tr.iloc[:,:-1]
df_tr_y = df_tr.iloc[:,-1]
df_ts_x = df_ts.iloc[:,:-1]
df_ts_y = df_ts.iloc[:,-1]

In [184]:
scaler = StandardScaler()
scaler.fit(df_tr_x)
df_tr_x = scaler.transform(df_tr_x)
scaler.fit(df_ts_x)
df_ts_x = scaler.transform(df_ts_x)

In [185]:
model = OneClassSVM(gamma='auto')

In [186]:
param_grid = {
    'kernel': ['rbf','poly','linear'],
    'nu': [0.001, 0.01, 0.5, 0.1, 0.2, 0.3]
}

In [187]:
grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=5, verbose=1)

In [188]:
grid_search.fit(df_tr_x)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=OneClassSVM(gamma='auto'),
             param_grid={'kernel': ['rbf', 'poly', 'linear'],
                         'nu': [0.001, 0.01, 0.5, 0.1, 0.2, 0.3]},
             scoring='f1_weighted', verbose=1)

In [189]:
best_params = grid_search.best_params_
print(f'Best Parmas:{best_params}')

Best Parmas:{'kernel': 'rbf', 'nu': 0.001}


In [190]:
svm_fit = OneClassSVM(**best_params, gamma='auto')
svm_fit.fit(df_tr_x)

OneClassSVM(gamma='auto', nu=0.001)

In [191]:
y_pred = svm_fit.predict(df_ts_x)

In [197]:
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [192]:
unique, counts = np.unique(y_pred, return_counts=True)
print (np.asarray((unique, counts)).T)

[[  -1    9]
 [   1 1991]]


In [193]:
df_ts_y = df_ts_y.reset_index()
fraud_pred = pd.DataFrame(y_pred)
fraud_pred= fraud_pred.rename(columns={0: 'prediction'})

In [194]:
TP = FN = FP = TN = 0
for j in range(len(df_tr_y)):
    if df_tr_y[j]== 0 and fraud_pred['prediction'][j] == 1:
        TP = TP+1
    elif df_tr_y[j]== 0 and fraud_pred['prediction'][j] == -1:
        FN = FN+1
    elif df_tr_y[j]== 1 and fraud_pred['prediction'][j] == 1:
        FP = FP+1
    else:
        TN = TN +1
print (TP,  FN,  FP,  TN)

1991 9 0 0


In [195]:
accuracy = (TP+TN)/(TP+FN+FP+TN)
print (accuracy)
sensitivity = TP/(TP+FN)
print (sensitivity)
specificity = TN/(TN+FP)
print (specificity)

0.9955
0.9955


ZeroDivisionError: division by zero

In [196]:
print(TP)
print(TN)
print(FP)
print(FN)

1991
0
0
9
